In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-29 13:57:53 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-29 13:57:53 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-29 13:57:53 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-29 13:58:08,945] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-29 13:58:09 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmp61dgcbr4/test.c -o /tmp/tmp61dgcbr4/test.o
2025-07-29 13:58:09 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmp61dgcbr4/test.o -laio -o /tmp/tmp61dgcbr4/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-29 13:58:09 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpb12sbb6s/test.c -o /tmp/tmpb12sbb6s/test.o
2025-07-29 13:58:09 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpb12sbb6s/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmpb12sbb6s/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-29 13:58:10 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-29 13:58:10 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-29 13:58:10 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-29 13:58:10 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-29 13:58:19 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-29 13:58:19 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-07-29 13:58:19 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-29 13:58:19 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-29 13:59:06 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-29 13:59:07 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [7]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>5<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [8]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-29 13:59:28 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-29 13:59:28 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-29 13:59:28 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-29 13:59:28 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-29 13:59:28 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-29 13:59:28 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-29 13:59:29 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
202

In [9]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-29 13:59:32 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

In [79]:
from src.selection.data import get_random_sample

sample = get_random_sample(
    people_by_category=people_by_category,
    mt=mt,
    attribute="politician",
    n_distractors=5
)
print(sample.prompt, ">>", sample.obj)

Which person from the following list has the occupation in common with Donald Trump?
Options: Nikola Jokić, Deontay Wilder, Ariana Grande, Mohamed Salah, Jake Tapper, Nikki Haley.
Ans: >> Nikki Haley


In [80]:
from src.functional import generate_with_patch, predict_next_token

# prompt_template = """Which of these peoples is by profession a {}?
# options: {}
# Answer:"""
# prompt = prompt_template.format(
#     sample.metadata["attribute"],
#     ", ".join(sample.options)
# )

prompt_template = """{}
Who among these people mentioned above is by profession a {}?
Answer:"""
prompt = prompt_template.format(
    ", ".join(sample.options),
    sample.metadata["attribute"]
)

print(prompt, " >> ")

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs=prompt,
)

Nikola Jokić, Deontay Wilder, Ariana Grande, Mohamed Salah, Jake Tapper, Nikki Haley
Who among these people mentioned above is by profession a politician?
Answer:  >> 
 Nikki Haley

Nikki Haley is an American politician who served as the 29th United States Ambassador


[[PredictedToken(token=' Nikki', prob=0.8203125, logit=20.125, token_id=70207, metadata=None),
  PredictedToken(token=' The', prob=0.04638671875, logit=17.25, token_id=578, metadata=None),
  PredictedToken(token=' Among', prob=0.019287109375, logit=16.375, token_id=22395, metadata=None),
  PredictedToken(token=' ', prob=0.01708984375, logit=16.25, token_id=220, metadata=None),
  PredictedToken(token=' Nikola', prob=0.00860595703125, logit=15.5625, token_id=97415, metadata=None)]]

In [81]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=[35, 47],
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------


[PredictedToken(token=' Nikki', prob=0.8203125, logit=20.125, token_id=70207, metadata=None),
 PredictedToken(token=' The', prob=0.04638671875, logit=17.25, token_id=578, metadata=None),
 PredictedToken(token=' Among', prob=0.019287109375, logit=16.375, token_id=22395, metadata=None),
 PredictedToken(token=' ', prob=0.01708984375, logit=16.25, token_id=220, metadata=None),
 PredictedToken(token=' Nikola', prob=0.00860595703125, logit=15.5625, token_id=97415, metadata=None)]

In [82]:
from src.attention import visualize_attn_matrix

###############################################################
N_KV_GROUPS = 1  # individual heads
# N_KV_GROUPS = mt.config.num_key_value_heads  # grouped heads
N_HEADS = mt.config.num_attention_heads
LAYER_RANGE = range(mt.n_layer)
###############################################################

group_id_to_head_indices = {
    i: [i * N_KV_GROUPS + j for j in range(N_KV_GROUPS)]
    for i in range(N_HEADS // N_KV_GROUPS)
}

In [83]:
# for group_id in group_id_to_head_indices:
layer_idx = 35
layer_name = mt.layer_name_format.format(layer_idx)
for group_id in [19, 46]:
    frm = group_id_to_head_indices[group_id][0]
    to = group_id_to_head_indices[group_id][-1]
    print(
        f"Layer: {layer_idx}, Heads: {group_id_to_head_indices[group_id]}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, frm: to+1]).mean(dim=0).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 35, Heads: [19]


Layer: 35, Heads: [46]


### Different Attribute

In [84]:
from src.selection.data  import load_people_by_category

people_by_nationality = load_people_by_category(
    tokenizer = mt.tokenizer,
    path = os.path.join(env_utils.DEFAULT_DATA_DIR, "selection_real/nationality.json")
)
list(people_by_nationality.keys())

2025-07-29 16:21:46 src.selection.data INFO     Loaded 16 categories


['Switzerland',
 'UAE',
 'Singapore',
 'Canada',
 'Mexico',
 'Brazil',
 'India',
 'Australia',
 'Israel',
 'South Africa',
 'United Kingdom',
 'Trinidad and Tobago',
 'New Zealand',
 'United States',
 'Belgium',
 'France']

In [85]:
from src.selection.data import get_random_sample

sample = get_random_sample(
    people_by_category=people_by_nationality,
    mt=mt,
    category="nationality",
    attribute="Australia",
    n_distractors=5
)
print(sample.prompt, ">>", sample.obj)

Which person from the following list has the nationality in common with Iggy Azalea?
Options: Catherine Deneuve, Nelson Mandela, Heath Ledger, Jean-Jacques Rousseau, Peter Minshall, Oprah Winfrey.
Ans: >> Heath Ledger


In [86]:
from src.functional import generate_with_patch, predict_next_token

prompt_template = """Which of these peoples is by profession a {}?
options: {}
Answer:"""
prompt = prompt_template.format(
    sample.metadata["attribute"],
    ", ".join(sample.options)
)

# prompt_template = """{}
# Who among these people mentioned above is from {}?
# Answer:"""
# prompt = prompt_template.format(
#     ", ".join(sample.options),
#     sample.metadata["attribute"]
# )

print(prompt, " >> ")

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs=prompt,
)

Which of these peoples is by profession a Australia?
options: Catherine Deneuve, Nelson Mandela, Heath Ledger, Jean-Jacques Rousseau, Peter Minshall, Oprah Winfrey
Answer:  >> 
 Heath Ledger
Explanation: Heath Ledger was an Australian actor and music video director. After performing roles in


[[PredictedToken(token=' Heath', prob=0.9296875, logit=21.375, token_id=48562, metadata=None),
  PredictedToken(token=' The', prob=0.0247802734375, logit=17.75, token_id=578, metadata=None),
  PredictedToken(token=' the', prob=0.006256103515625, logit=16.375, token_id=279, metadata=None),
  PredictedToken(token=' d', prob=0.004302978515625, logit=16.0, token_id=294, metadata=None),
  PredictedToken(token=' Australia', prob=0.0026092529296875, logit=15.5, token_id=8494, metadata=None)]]

In [88]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=[35, 47],
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------


[PredictedToken(token=' Heath', prob=0.92578125, logit=21.375, token_id=48562, metadata=None),
 PredictedToken(token=' The', prob=0.0279541015625, logit=17.875, token_id=578, metadata=None),
 PredictedToken(token=' the', prob=0.007049560546875, logit=16.5, token_id=279, metadata=None),
 PredictedToken(token=' d', prob=0.0040283203125, logit=15.9375, token_id=294, metadata=None),
 PredictedToken(token=' Australia', prob=0.0027618408203125, logit=15.5625, token_id=8494, metadata=None)]

In [89]:
# for group_id in group_id_to_head_indices:
layer_idx = 35
layer_name = mt.layer_name_format.format(layer_idx)
for group_id in [19, 46]:
    frm = group_id_to_head_indices[group_id][0]
    to = group_id_to_head_indices[group_id][-1]
    print(
        f"Layer: {layer_idx}, Heads: {group_id_to_head_indices[group_id]}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, frm: to+1]).mean(dim=0).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 35, Heads: [19]


Layer: 35, Heads: [46]


In [135]:
import random

# options = ["Table", "Orange", "Transistor", "Spinach", "Piano", "Coffee"]
# prompt_template = """Which of these objects is by type a {}?
# options: {}
# Answer:"""
# prompt = prompt_template.format(
#     "musical instrument",
#     ", ".join(options)
# )
# print(prompt)

# options = ["anaconda", "python", "cobra", "viper", "mamba", "rattlesnake"]
# prompt = f"""Which of these snake names is also a programming language?
# options: {", ".join(options)}
# Answer:"""

options = ["The Space Needle", "Louvre Museum", "Colosseum", "State of Liberty", "Big Ben", "Christ the Redeemer"]
# options += ["Eiffel Tower"]
# random.shuffle(options)
country = "France"
prompt_template = """Which of these landmarks is located in {}?
options:\n{}
Answer:"""
prompt = prompt_template.format(
    country,
    "\n".join([f"{idx+1}. {opt}" for idx, opt in enumerate(options)])
)

print(prompt)

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
# predict_next_token(
#     mt=mt,
#     inputs=prompt,
# )

Which of these landmarks is located in France?
options:
1. The Space Needle
2. Louvre Museum
3. Colosseum
4. State of Liberty
5. Big Ben
6. Christ the Redeemer
Answer:
 The Louvre Museum. The Louvre, or the Louvre Museum, is the world's largest


In [136]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=[35, 47],
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------


[PredictedToken(token=' ', prob=0.470703125, logit=21.25, token_id=220, metadata=None),
 PredictedToken(token=' The', prob=0.28515625, logit=20.75, token_id=578, metadata=None),
 PredictedToken(token=' Lou', prob=0.1533203125, logit=20.125, token_id=9928, metadata=None),
 PredictedToken(token=' \n', prob=0.0341796875, logit=18.625, token_id=720, metadata=None),
 PredictedToken(token=' the', prob=0.01104736328125, logit=17.5, token_id=279, metadata=None)]

In [137]:
# for group_id in group_id_to_head_indices:
layer_idx = 35
layer_name = mt.layer_name_format.format(layer_idx)
for group_id in [19, 46]:
    frm = group_id_to_head_indices[group_id][0]
    to = group_id_to_head_indices[group_id][-1]
    print(
        f"Layer: {layer_idx}, Heads: {group_id_to_head_indices[group_id]}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, frm: to+1]).mean(dim=0).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 35, Heads: [19]


Layer: 35, Heads: [46]


In [141]:
options = ["Marie Curie", "Albert Einstein", "Grover Cleveland", "Charles Darwin", "Nikola Tesla", "Issac Newton"]
prompt_template = """Which of these people is not a scientist?
options: {}
Answer:"""
prompt = prompt_template.format(
    ", ".join(options)
)
print(prompt)

predict_next_token(
    mt=mt,
    inputs=prompt,
)

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=[35, 47],
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

Which of these people is not a scientist?
options: Marie Curie, Albert Einstein, Grover Cleveland, Charles Darwin, Nikola Tesla, Issac Newton
Answer:
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------


[PredictedToken(token=' Gro', prob=0.96875, logit=21.5, token_id=18370, metadata=None),
 PredictedToken(token=' The', prob=0.00653076171875, logit=16.5, token_id=578, metadata=None),
 PredictedToken(token=' ', prob=0.002716064453125, logit=15.625, token_id=220, metadata=None),
 PredictedToken(token='Gro', prob=0.0023956298828125, logit=15.5, token_id=77435, metadata=None),
 PredictedToken(token=' the', prob=0.00154876708984375, logit=15.0625, token_id=279, metadata=None)]

In [142]:
# for group_id in group_id_to_head_indices:
layer_idx = 35
layer_name = mt.layer_name_format.format(layer_idx)
for group_id in [19, 46]:
    frm = group_id_to_head_indices[group_id][0]
    to = group_id_to_head_indices[group_id][-1]
    print(
        f"Layer: {layer_idx}, Heads: {group_id_to_head_indices[group_id]}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, frm: to+1]).mean(dim=0).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 35, Heads: [19]


Layer: 35, Heads: [46]


In [150]:
options = ["Michael Jordan", "Serena Williams", "Mike Pence", "Issac Newton", "Tom Cruise"]
prompt_template = """Which of these people is a politician or a scientist?
options: {}
Answer:"""

prompt = prompt_template.format(
    ", ".join(options)
)
print(prompt)
gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=[35, 47],
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

Which of these people is a politician or a scientist?
options: Michael Jordan, Serena Williams, Mike Pence, Issac Newton, Tom Cruise
Answer:
 Mike Pence
Mike Pence is a politician, specifically the 48th Vice President of the United States
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------


[PredictedToken(token=' Mike', prob=0.78125, logit=20.25, token_id=11519, metadata=None),
 PredictedToken(token=' Iss', prob=0.056640625, logit=17.625, token_id=16314, metadata=None),
 PredictedToken(token=' The', prob=0.0498046875, logit=17.5, token_id=578, metadata=None),
 PredictedToken(token=' the', prob=0.0086669921875, logit=15.75, token_id=279, metadata=None),
 PredictedToken(token=' ', prob=0.0086669921875, logit=15.75, token_id=220, metadata=None)]

In [151]:
# for group_id in group_id_to_head_indices:
layer_idx = 35
layer_name = mt.layer_name_format.format(layer_idx)
for group_id in [19, 46]:
    frm = group_id_to_head_indices[group_id][0]
    to = group_id_to_head_indices[group_id][-1]
    print(
        f"Layer: {layer_idx}, Heads: {group_id_to_head_indices[group_id]}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, frm: to+1]).mean(dim=0).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 35, Heads: [19]


Layer: 35, Heads: [46]
